## Test AWA Environment/Interface Plugins

In [1]:
from plugins.environments.awa_environment import AWAEnvironment
from plugins.interfaces.test_awa_interface import TestAWAInterface

env = AWAEnvironment(interface=TestAWAInterface(), variables={"Q6:BCTRL":[-1,1]})

In [2]:
env.dict()

{'name': 'awa_environment',
 'interface': {'name': 'awa_interface',
  'params': {},
  'camera_app': <plugins.interfaces.camera.AWACamera at 0x1c2c099c370>,
  'x': None},
 'variables': {'Q6:BCTRL': [-1.0, 1.0]},
 'observables': ['AWAICTMon:Ch1',
  'AWAICTMon:Ch2',
  'AWAICTMon:Ch3',
  'AWAICTMon:Ch4',
  'YAG1:XRMS'],
 'params': {},
 'target_charge_PV': 'AWAICTMon:Ch1',
 'target_charge': None,
 'fractional_charge_deviation': 0.1}

In [3]:
env.set_variables({"Q6:BCTRL": 0.5})

In [4]:
env.get_observables(["YAG1:XRMS"])

{'YAG1:XRMS': 0.25, 'AWAICTMon:Ch1': 1.0}

## Test with Xopt BE

In [5]:
from xopt import Xopt, Evaluator, VOCS
from xopt.generators.bayesian import BayesianExplorationGenerator

def evaluate(inputs):
    env.set_variables(inputs)
    return env.get_observables(env.observables)

vocs = VOCS(variables=env.variables, objectives={"YAG1:XRMS":"MINIMIZE"})
generator = BayesianExplorationGenerator(vocs)
evaluator = Evaluator(function=evaluate)
X = Xopt(vocs=vocs, evaluator=evaluator, generator=generator)

In [6]:
vocs.dict()

{'variables': {'Q6:BCTRL': [-1.0, 1.0]},
 'constraints': {},
 'objectives': {'YAG1:XRMS': 'MINIMIZE'},
 'constants': {},
 'linked_variables': {}}

In [7]:
X.step()

In [8]:
X.data

,Q6:BCTRL,AWAICTMon:Ch1,YAG1:XRMS,xopt_runtime,xopt_error
1,-0.490652,1.0,0.240740,0.000056,False
2,-0.509003,1.0,0.259084,0.000019,False
3,-0.142317,1.0,0.020254,0.000016,False


In [9]:
for i in range(5):
    X.step()

In [10]:
X.data

,Q6:BCTRL,AWAICTMon:Ch1,YAG1:XRMS,xopt_runtime,xopt_error
1,-0.490652,1.0,0.240740,0.000056,False
2,-0.509003,1.0,0.259084,0.000019,False
3,-0.142317,1.0,0.020254,0.000016,False
4,1.000000,1.0,1.000000,0.000042,False
5,-1.000000,1.0,1.000000,0.000050,False
6,0.486926,1.0,0.237097,0.000045,False
7,0.189031,1.0,0.035733,0.000044,False
8,0.778734,1.0,0.606426,0.000046,False


In [ ]:
X.data.plot(x="Q6:BCTRL", y="YAG1:XRMS")